In [1]:
import os
from itertools import product
from astropy.io import fits
from astropy.table import Table

from lvmdrp import path, __version__ as drpver
from lvmdrp.utils import metadata as md
from lvmdrp.functions import run_drp as drp
from lvmdrp.functions import imageMethod as image_tasks
from lvmdrp.functions import rssMethod as rss_tasks
from lvmdrp.core.constants import SPEC_CHANNELS

# Instructions

This notebook implementes a quick reduction of the LVM data. Main asumptions are:

1. There is a `data_calib` directory containing master calibration frames for:
    - pixel mask (`lvm-mpixmask-{CAMERA}.fits`)
    - bias (`lvm-mbias-{CAMERA}.fits`)
    - dark (`lvm-mdark-{CAMERA}.fits`)
    - traces (`traces_{CAMERA}_p4.fits`)
    - wavelengths (`lvm-mwave_neon-{CAMERA}.fits`)
    - LSF (`lvm-mlsf_neon-{CAMERA}.fits`)

2. There is a `data_science` directory containing the **raw** target science exposures (`sdR-s-{CAMERA}-{EXPNUM:>08d}.fits.gz`)

3. Data products from this reduction can be stored in a directory `data_products` (this directory will be created by this notebook if it does not exists).

This reduction will push raw frames from preprocessing down to wavelength calibration.

In [ ]:
# # define paths
CALDIR = "data_calib/"
SCIDIR = "data_science/"
OUTDIR = "data_products/"
os.makedirs(OUTDIR, exist_ok=True)

# # define cameras
SPECIDS = (1,2,3)
CHANNELS = "brz"
CAMERAS = [f"{channel}{spec}" for channel, spec in product(CHANNELS, SPECIDS)]

In [ ]:
# target science directory
sci_paths = sorted([os.path.join(SCIDIR, sci_name) for sci_name in os.listdir(SCIDIR)])
print(sci_paths)

In [ ]:
mjd = 60145
tileid = 1111
expnum = 1361

In [ ]:
sci_metadata = md.get_metadata(tileid=tileid, mjd=mjd, expnum=expnum).sort_values(["expnum", "camera"])

arc_lamps = {"b": "hgne", "r": "neon", "z": "neon"}

spec_paths = []
for sci in sci_metadata.to_dict("records"):
    
    # define sci paths
    sci_path = path.full("lvm_raw", camspec=sci["camera"], **sci)
    psci_path = path.full("lvm_anc", drpver=drpver, kind="p", imagetype=sci["imagetyp"], **sci)
    dsci_path = path.full("lvm_anc", drpver=drpver, kind="d", imagetype=sci["imagetyp"], **sci)
    xsci_path = path.full("lvm_anc", drpver=drpver, kind="x", imagetype=sci["imagetyp"], **sci)
    wsci_path = path.full("lvm_anc", drpver=drpver, kind="w", imagetype=sci["imagetyp"], **sci)
    # lamps configuration per spectrograph channel
    lamps = arc_lamps[sci["camera"][0]]
    
    # define calibration frames paths
    masters = md.match_master_metadata(target_mjd=sci["mjd"], target_camera=sci["camera"], target_imagetyp=sci["imagetyp"])
    mpixmask_path = path.full("lvm_master", drpver=drpver, kind="mpixmask", **masters["pixmask"].to_dict())
    mbias_path = path.full("lvm_master", drpver=drpver, kind="mbias", **masters["bias"].to_dict())
    mdark_path = path.full("lvm_master", drpver=drpver, kind="mdark", **masters["dark"].to_dict())
    mtrace_path = path.full("lvm_master", drpver=drpver, kind="mtrace", **masters["trace"].to_dict())
    mwave_path = path.full("lvm_master", drpver=drpver, kind=f"mwave_{lamps}", **masters["wave"].to_dict())
    mlsf_path = path.full("lvm_master", drpver=drpver, kind=f"mlsf_{lamps}", **masters["lsf"].to_dict())
    mflat_path = path.full("lvm_master", drpver=drpver, kind="mfiberflat", **masters["fiberflat"].to_dict())
    
    # if os.path.isfile(wsci_path):
    #     print(f"skipping {wsci_path}, file already exist")
    #     continue
    
    # preprocess frame
    image_tasks.preproc_raw_frame(in_image=sci_path, out_image=psci_path, in_mask=mpixmask_path)
    
    # detrend frame
    image_tasks.detrend_frame(in_image=psci_path, out_image=dsci_path, in_bias=mbias_path, in_dark=mdark_path, in_slitmap=Table(drp.fibermap.data))
    
    # extract 1d spectra
    image_tasks.extract_spectra(in_image=dsci_path, out_rss=xsci_path, in_trace=mtrace_path, method="aperture", aperture=3)
    
    # wavelength calibrate & resample
    iwave, fwave = SPEC_CHANNELS[sci["camera"][0]]
    rss_tasks.create_pixel_table(in_rss=xsci_path, out_rss=wsci_path, arc_wave=mwave_path, arc_fwhm=mlsf_path)
    rss_tasks.resample_wavelength(in_rss=wsci_path, out_rss=wsci_path, method="linear", disp_pix=0.5, start_wave=iwave, end_wave=fwave, err_sim=10, parallel=0)
    
    # apply fiberflat correction
    rss_tasks.apply_fiberflat(in_rss=wsci_path, out_rss=wsci_path, in_flat=mflat_path)
    
    # list paths for spectrograph combination
    spec_paths.append(wsci_path)

In [ ]:
specid = 1

jsci_path = os.path.join(OUTDIR, f"lvm-object-sp{specid}-{expnum:>08d}.fits")
_ = rss_tasks.join_spec_channels(spec_paths[specid-1::3], out_rss=jsci_path)

In [ ]:
from lvmdrp.core import rss
import matplotlib.pyplot as plt

sci = rss.RSS()
sci.loadFitsData(jsci_path)
sci.apply_pixelmask()

plt.figure(figsize=(20,7))
plt.step(sci._wave, sci._data[319], lw=1, color="tab:red")
# plt.gca().set_yscale("log")